In [37]:
import pandas as pd

In [38]:
import numpy as np

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
#to clarify, the columns targeted are going to be 'goals_home', 'goals_away', 'result_home'

In [41]:
import tensorflow as tf
from tensorflow import keras

In [42]:
tf.autograph.experimental.do_not_convert

<function tensorflow.python.autograph.impl.api.do_not_convert(func=None)>

In [43]:
all_merged = pd.read_csv('/Users/mikhail/Jupyter/Jupyter-1/Projects/all_merged.csv')
all_merged_copy = all_merged.copy()

In [44]:
output = all_merged[['goals_home','goals_away','result_home']]

In [45]:
all_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2660 entries, 0 to 2659
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          2660 non-null   int64  
 1   gameID              2660 non-null   int64  
 2   homeTeamID          2660 non-null   int64  
 3   awayTeamID          2660 non-null   int64  
 4   homeProbability     2660 non-null   float64
 5   drawProbability     2660 non-null   float64
 6   awayProbability     2660 non-null   float64
 7   homeGoalsHalfTime   2660 non-null   int64  
 8   awayGoalsHalfTime   2660 non-null   int64  
 9   teamID_home         2660 non-null   int64  
 10  goals_home          2660 non-null   int64  
 11  xGoals_home         2660 non-null   float64
 12  shots_home          2660 non-null   int64  
 13  shotsOnTarget_home  2660 non-null   int64  
 14  deep_home           2660 non-null   int64  
 15  ppda_home           2660 non-null   float64
 16  fouls_

In [46]:
all_merged = all_merged.drop(columns=['goals_home','goals_away','result_home','gameID'])

In [47]:
X_train_full, X_test, y_train_full, y_test = train_test_split(all_merged,output)

In [48]:
X_train,X_valid, y_train, y_valid = train_test_split(X_train_full,y_train_full)

In [49]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(300,activation='tanh')(input_)
hidden2 = keras.layers.Dense(200,activation='tanh')(hidden1)
hidden3 = keras.layers.Dense(100,activation='tanh')(hidden2)
output_hg = keras.layers.Dense(1,name ='output_hg')(hidden3)
output_ag = keras.layers.Dense(1,name ='output_ag')(hidden3)
output_r = keras.layers.Dense(1,name ='output_r')(hidden3)
model = keras.Model(inputs = [input_],outputs=[output_ag,output_hg,output_r])

In [50]:
model.compile(loss=['mse','mse','mse'],loss_weights=[0.2,0.2,0.6],optimizer='sgd')


In [51]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

history = model.fit(X_train,[y_train['goals_home'],y_train['goals_away'],y_train['result_home']],epochs = 20,
validation_data=(X_valid,[y_valid['goals_home'],y_valid['goals_away'],y_valid['result_home']]),batch_size=128,callbacks=early_stopping_cb)

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 1/12 [=>............................] - ETA: 2s - loss: 2.2032 - output_ag_loss: 2.1631 - output_hg_loss: 3.0563 - output_r_loss: 1.9323WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x174c62820> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the 

In [52]:
model.get_weights()

[array([[-0.11973582, -0.06234781,  0.01115592, ..., -0.03891729,
         -0.07107595, -0.09159345],
        [ 0.12179596,  0.02619457, -0.07391318, ..., -0.12406266,
         -0.01837176, -0.04399824],
        [ 0.08553202, -0.09595888,  0.0993774 , ..., -0.01243388,
         -0.03663001, -0.05189254],
        ...,
        [-0.07562255,  0.08274058, -0.07067646, ..., -0.07823381,
         -0.0168925 ,  0.00077742],
        [ 0.10038292,  0.03593196, -0.03419323, ..., -0.06297732,
          0.0227116 ,  0.02264117],
        [ 0.08969574,  0.01977812, -0.09140966, ..., -0.09931087,
         -0.1213953 , -0.05748292]], dtype=float32),
 array([-3.34679462e-05,  0.00000000e+00, -7.96092572e-05, -1.55366379e-05,
         1.04960898e-04,  0.00000000e+00,  1.63678724e-05, -4.39347714e-05,
         0.00000000e+00,  0.00000000e+00, -1.92788648e-05,  1.68434201e-06,
         2.13830458e-06,  0.00000000e+00,  7.58229999e-06,  3.78617988e-05,
        -4.01178022e-06, -4.01414982e-05, -7.66321173e

In [55]:
total_loss, hg_loss, ag_loss, r_loss = model.evaluate(X_test,[y_test['goals_home'],y_test['goals_away'],y_test['result_home']])

21/21 [==============================] - 0s 404us/step - loss: inf - output_ag_loss: inf - output_hg_loss: inf - output_r_loss: inf      


In [54]:
history.history

{'loss': [1.8479000329971313,
  1.0807349681854248,
  1.0760746002197266,
  1.0874632596969604,
  1.0814034938812256,
  1.0748424530029297,
  1.0695383548736572,
  1.0732693672180176,
  1.0713258981704712,
  1.0655176639556885,
  1.076080322265625,
  1.0660383701324463,
  1.068642020225525,
  1.0732039213180542,
  1.0691745281219482,
  1.0761967897415161,
  1.0702261924743652,
  1.0702100992202759],
 'output_ag_loss': [1.7816050052642822,
  1.6719074249267578,
  1.6651966571807861,
  1.667375087738037,
  1.6787219047546387,
  1.6620434522628784,
  1.6601412296295166,
  1.6682218313217163,
  1.6542699337005615,
  1.6471235752105713,
  1.6561709642410278,
  1.653922200202942,
  1.6546343564987183,
  1.6443614959716797,
  1.6562235355377197,
  1.656302571296692,
  1.6464860439300537,
  1.652064323425293],
 'output_hg_loss': [1.685749888420105,
  1.4880675077438354,
  1.4848408699035645,
  1.5004701614379883,
  1.4843143224716187,
  1.4811047315597534,
  1.4817965030670166,
  1.48134541511